# Segmentation and Binning of single nuclei from Visium HD
Adpated from https://www.10xgenomics.com/analysis-guides/segmentation-visium-hd

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata
import geopandas as gpd
import scanpy as sc

import pathlib

from tifffile import imread, imwrite
from csbdeep.utils import normalize
from stardist.models import StarDist2D
from shapely.geometry import Polygon, Point
from scipy import sparse
from matplotlib.colors import ListedColormap

import nuclei_segmentation_plotting as nsp

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
img_name = 'B23.1556.tif'

In [ ]:
path_to_data = pathlib.Path('..') / 'data' / 'raw'
path_to_intermediate_data = path_to_data.parent / 'intermediate'
path_to_intermediate_data.mkdir(parents = True, exist_ok=True)
result_dir = pathlib.Path('..') / 'results'
result_dir.mkdir(parents = True, exist_ok = True)

In [ ]:
rois = dict()
rois['top_left'] = (11591,8072,17200,12176)
rois['bottom_right'] = (23288,15802,27803,19290)
rois['bottom_right_enlarged'] = (22696,15767,27742,19310)
rois['top_right']= (21373,8345,26435,12244)
rois['middle'] = (14822,13042,20042,16829)


segmentation_qc_rois = dict()
segmentation_qc_rois['high_density'] = (22887,16780,23207,17100)
segmentation_qc_rois['low_density'] = (25366,10611,26301,11443)
segmentation_qc_rois['medium_density'] = (26825,16530,27354,16990)
segmentation_qc_rois['mixed_density'] = (16022,10920,16857,11644)
segmentation_qc_rois['middle_small_area'] = (16576,13474,17383,14212)

In [ ]:
img = imread(path_to_data / img_name)

# Load the pretrained model
model = StarDist2D.from_pretrained('2D_versatile_he')

# Percentile normalization of the image
# Adjust min_percentile and max_percentile as needed
min_percentile = 2
max_percentile = 98
img = normalize(img, min_percentile, max_percentile)

# Nuclei Mask and GeoDataframe Creation

In [ ]:
# Predict cell nuclei using the normalized image
# Adjust nms_thresh and prob_thresh as needed

labels, polys = model.predict_instances_big(img, axes='YXC', block_size=4096, prob_thresh=0.1, nms_thresh=0.001, min_overlap=128, context=128, normalizer=None, n_tiles=(4,4,1)) # original: nms_thresh=0.001, prob_thresh=0.01


In [ ]:
# Creating a list to store Polygon geometries
geometries = []

# Iterating through each nuclei in the 'polys' DataFrame
for nuclei in range(len(polys['coord'])):

    # Extracting coordinates for the current nuclei and converting them to (y, x) format
    coords = [(y, x) for x, y in zip(polys['coord'][nuclei][0], polys['coord'][nuclei][1])]

    # Creating a Polygon geometry from the coordinates
    geometries.append(Polygon(coords))

# Creating a GeoDataFrame using the Polygon geometries
gdf = gpd.GeoDataFrame(geometry=geometries)
gdf['id'] = [f"ID_{i+1}" for i, _ in enumerate(gdf.index)]


In [ ]:
img.shape

# Fig 2A

In [ ]:
cmap=ListedColormap(['grey'])



for name, coords in segmentation_qc_rois.items():
    nsp.plot_mask_and_save_image(title=name,gdf=gdf,bbox=coords,cmap=cmap,img=img,output_name=result_dir / "image_mask_{}.tif".format(name))


# Binning Visium HD Gene Expression Data  
In the following code block, we load the 2x2 µm Visium HD gene expression data and tissue position information. A GeoDataframe is then created. Be sure that the expression data and tissue position files are in the same directory as the high-resolution H&E microscope image.

In [ ]:
# Load Visium HD data
raw_h5_file = path_to_data / 'square_002um' / 'filtered_feature_bc_matrix.h5'
adata = sc.read_10x_h5(raw_h5_file)

# Load the Spatial Coordinates
tissue_position_file = path_to_data / 'square_002um' / 'spatial' / 'tissue_positions.parquet'
df_tissue_positions=pd.read_parquet(tissue_position_file)

#Set the index of the dataframe to the barcodes
df_tissue_positions = df_tissue_positions.set_index('barcode')

# Create an index in the dataframe to check joins
df_tissue_positions['index']=df_tissue_positions.index

# Adding the tissue positions to the meta data
adata.obs =  pd.merge(adata.obs, df_tissue_positions, left_index=True, right_index=True)

# Create a GeoDataFrame from the DataFrame of coordinates
geometry = [Point(xy) for xy in zip(df_tissue_positions['pxl_col_in_fullres'], df_tissue_positions['pxl_row_in_fullres'])]
gdf_coordinates = gpd.GeoDataFrame(df_tissue_positions, geometry=geometry)


We next check each barcode to determine if they are in a cell nucleus. There is a chance that two or more cell nuclei can overlap, so to remove barcode assignment ambiguity, we will later filter to retain only barcodes that are uniquely assigned.

In [ ]:
# Perform a spatial join to check which coordinates are in a cell nucleus
result_spatial_join = gpd.sjoin(gdf_coordinates, gdf, how='left', predicate='within')

# Identify nuclei associated barcodes and find barcodes that are in more than one nucleus
result_spatial_join['is_within_polygon'] = ~result_spatial_join['index_right'].isna()
barcodes_in_overlaping_polygons = pd.unique(result_spatial_join[result_spatial_join.duplicated(subset=['index'])]['index'])
result_spatial_join['is_not_in_an_polygon_overlap'] = ~result_spatial_join['index'].isin(barcodes_in_overlaping_polygons)

# Remove barcodes in overlapping nuclei
barcodes_in_one_polygon = result_spatial_join[result_spatial_join['is_within_polygon'] & result_spatial_join['is_not_in_an_polygon_overlap']]

# The AnnData object is filtered to only contain the barcodes that are in non-overlapping polygon regions
filtered_obs_mask = adata.obs_names.isin(barcodes_in_one_polygon['index'])
filtered_adata = adata[filtered_obs_mask,:]

# Add the results of the point spatial join to the Anndata object
filtered_adata.obs =  pd.merge(filtered_adata.obs, barcodes_in_one_polygon[['index','geometry','id','is_within_polygon','is_not_in_an_polygon_overlap']], left_index=True, right_index=True)


Next we perform a gene-wise count summation for the custom binned data. This step will take a few minutes to execute and can vary based on the number of nuclei in the image.

In [ ]:
# Group the data by unique nucleous IDs
groupby_object = filtered_adata.obs.groupby(['id'], observed=True)

# Extract the gene expression counts from the AnnData object
counts = filtered_adata.X

# Obtain the number of unique nuclei and the number of genes in the expression data
N_groups = groupby_object.ngroups
N_genes = counts.shape[1]

# Initialize a sparse matrix to store the summed gene counts for each nucleus
summed_counts = sparse.lil_matrix((N_groups, N_genes))

# Lists to store the IDs of polygons and the current row index
polygon_id = []
row = 0

# Iterate over each unique polygon to calculate the sum of gene counts.
for polygons, idx_ in groupby_object.indices.items():
    summed_counts[row] = counts[idx_].sum(0)
    row += 1
    polygon_id.append(polygons)

# Create and AnnData object from the summed count matrix
summed_counts = summed_counts.tocsr()
grouped_filtered_adata = anndata.AnnData(X=summed_counts,obs=pd.DataFrame(polygon_id,columns=['id'],index=polygon_id),var=filtered_adata.var)

%store grouped_filtered_adata


# Nuclei Binning Results  
In this demonstration, we filter the results to remove very large nuclei, which could be improperly segmented nuclei aggregates or image artifacts, and remove nuclei with too few UMI counts to make cluster interpretation and visualization easier. This step is optional and performing it will likely depend on the sample.

In [ ]:
# Store the area of each nucleus in the GeoDataframe
gdf['area'] = gdf['geometry'].area

sc.pp.calculate_qc_metrics(grouped_filtered_adata, inplace=True)

# Plot the nuclei area distribution before and after filtering
nsp.plot_nuclei_area(gdf=gdf,area_cut_off=1000)


Based on the nuclei distribution we selected a value of 500 to filter the data.

Next we plot the total UMI distribution.

In [ ]:
# Plot total UMI distribution
nsp.total_umi(grouped_filtered_adata, 50) # original value: 100


For this dataset, a total UMI cutoff of 100 was used. These values (nuclei area and UMI cutoff) may need adjustment for other datasets. Selecting appropriate cutoff values might require iteration based on the intepretation of the clustering results.

The following code block filters the data.

In [ ]:
# Create a mask based on the 'id' column for values present in 'gdf' with 'area' less than max_area
max_area = 800
mask_area = grouped_filtered_adata.obs['id'].isin(gdf[gdf['area'] < max_area].id)

# Create a mask based on the 'total_counts' column for values greater than 100
min_total_counts = 50 # original value: 100
mask_count = grouped_filtered_adata.obs['total_counts'] > min_total_counts

# Apply both masks to the original AnnData to create a new filtered AnnData object
count_area_filtered_adata = grouped_filtered_adata[mask_area & mask_count, :]

# Calculate quality control metrics for the filtered AnnData object
sc.pp.calculate_qc_metrics(count_area_filtered_adata, inplace=True)


# View filtered mask

In [ ]:
gdf[gdf['area'] < max_area]

In [ ]:
# Plot the nuclei segmentation
cmap=ListedColormap(['grey'])
for name, coords in segmentation_qc_rois.items():
    nsp.plot_mask_and_save_image(title=name,gdf=gdf[gdf['area'] < max_area],bbox=coords,cmap=cmap,img=img,output_name=result_dir / "image_mask_filtered_{}.tif".format(name))


In [ ]:
for name, coords in rois.items():
    nsp.plot_mask_and_save_image(title=name,gdf=gdf[gdf['area'] < max_area],bbox=coords,cmap=cmap,img=img,output_name=result_dir / "image_mask_filtered_{}.tif".format(name))

To assess the binning results, we examine gene expression and clustering. In the next block of code, the results undergo clustering. It is important to note that the resolution parameter used in Leiden clustering may need adjustment when working with different datasets.

In [ ]:
# Normalize total counts for each cell in the AnnData object
sc.pp.normalize_total(count_area_filtered_adata, inplace=True)

# Logarithmize the values in the AnnData object after normalization
sc.pp.log1p(count_area_filtered_adata)

In [ ]:
# Identify highly variable genes in the dataset using the Seurat method
sc.pp.highly_variable_genes(count_area_filtered_adata, flavor="seurat", n_top_genes=2000)
sc.pp.pca(count_area_filtered_adata)

# Build a neighborhood graph based on PCA components
sc.pp.neighbors(count_area_filtered_adata)

In [ ]:
# Adjust the resolution parameter as needed for different samples
sc.tl.leiden(count_area_filtered_adata)

When the clustering results are plotted, we see clusters that align with morphological features (Figure 6)



# Save adata and geodataframe

In [ ]:
count_area_filtered_adata.write(path_to_intermediate_data /'count_area_filtered_adata.h5ad')

In [ ]:
gdf.to_parquet(path_to_intermediate_data / 'geodataframe.parquet')